In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import json
import pandas as pd
import logging
from ipywidgets import Dropdown

<IPython.core.display.Javascript object>

In [3]:
import boto3
import sagemaker

/home/thangtran3112/aws-sage-maker/machine-learning/venv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[01/12/25 12:20:03] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=230742;file:///home/thangtran3112/aws-sage-maker/machine-learning/venv/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=771874;file:///home/thangtran3112/aws-sage-maker/machine-learning/venv/lib/python3.12/site-packages/botocore/credentials.py#1278\1278]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/thangtran3112/.config/sagemaker/config.yaml


<IPython.core.display.Javascript object>

In [4]:
boto3.set_stream_logger(name="botocore.credentials", level=logging.WARNING)

<IPython.core.display.Javascript object>

In [5]:
sess = sagemaker.Session()
region = sess.boto_region_name
print(region)

us-west-2


<IPython.core.display.Javascript object>

In [31]:
# download JumpStart model_manifest file. https://docs.aws.amazon.com/sagemaker/latest/dg/jumpstart-deploy.html
boto3.client("s3").download_file(
    f"jumpstart-cache-prod-{region}", "models_manifest.json", "models_manifest.json"
)
with open("models_manifest.json", "rb") as json_file:
    model_list = json.load(json_file)

print("number of models: ", len(model_list))

number of models:  10433


<IPython.core.display.Javascript object>

In [32]:
model_df = pd.DataFrame(model_list)
model_df.sample(20)

,model_id,version,min_version,spec_key,search_keywords
4626,meta-textgeneration-llama-codellama-7b-python,2.1.0,2.198.0,community_models/meta-textgeneration-llama-cod...,NaN
4237,meta-textgeneration-llama-3-1-8b,2.4.0,2.232.1,community_models/meta-textgeneration-llama-3-1...,NaN
8153,tensorflow-ic-swin-tiny-patch4-window7-224,2.0.4,2.189.0,community_models/tensorflow-ic-swin-tiny-patch...,NaN
1029,huggingface-llm-openlm-research-open-llama-7b-v2,1.1.3,2.189.0,community_models/huggingface-llm-openlm-resear...,NaN
3151,huggingface-txt2img-fictiverse-stable-diffusio...,2.0.5,2.189.0,community_models/huggingface-txt2img-fictivers...,"[Text, Text to image, Foundation Models]"
9549,tensorflow-od1-ssd-resnet50-v1-fpn-640x640-coc...,1.1.0,2.129.0,community_models/tensorflow-od1-ssd-resnet50-v...,NaN
3798,huggingface-zstc-emrecan-distilbert-base-turki...,2.0.3,2.189.0,community_models/huggingface-zstc-emrecan-dist...,NaN
5875,pytorch-ic-wide-resnet50-2,1.0.0,2.75.0,community_models/pytorch-ic-wide-resnet50-2/sp...,NaN
5163,mxnet-od-ssd-512-vgg16-atrous-coco,2.0.5,2.189.0,community_models/mxnet-od-ssd-512-vgg16-atrous...,"[Vision, Object Detection]"
4227,meta-textgeneration-llama-3-1-70b-instruct,2.2.3,2.225.0,community_models/meta-textgeneration-llama-3-1...,NaN


<IPython.core.display.Javascript object>

In [34]:
# filter-out all the Object Detection models from the manifest list.
od_models = []
for model in model_list:
    model_id = model["model_id"]
    if ("-od-" in model_id or "-od1-" in model_id) and model_id not in od_models:
        od_models.append(model_id)

print(f"Number of od models available for inference: {len(od_models)}")

Number of od models available for inference: 70


<IPython.core.display.Javascript object>

In [35]:
pytorch_nvidia_models = []
for model_id in od_models:
    if "nvidia" in model_id:
        pytorch_nvidia_models.append(model_id)
pytorch_nvidia_models

['pytorch-od-nvidia-ssd']

<IPython.core.display.Javascript object>

In [36]:
# display the model-ids in a dropdown to select a model for inference.
infer_model_dropdown = Dropdown(
    options=od_models,
    value=pytorch_nvidia_models[0],
    description="Select a model:",
    style={"description_width": "initial"},
    layout={"width": "max-content"},
)

<IPython.core.display.Javascript object>

## Not all models can be fine-tuned or transfer learning
For the list of fine-tunable models, please visit: https://sagemaker.readthedocs.io/en/stable/doc_utils/pretrainedmodels.html#built-in-algorithms-with-pre-trained-model-table

In [24]:
display(infer_model_dropdown)


Dropdown(description='Select a model:', index=17, layout=Layout(width='max-content'), options=('mxnet-od-faste…

<IPython.core.display.Javascript object>

In [25]:
print(infer_model_dropdown.value)

pytorch-od-nvidia-ssd


<IPython.core.display.Javascript object>

In [38]:
# filter-out all the Image Classification models from the manifest list.
image_classification_models = []
for model in model_list:
    model_id = model["model_id"]
    if ("-ic-" in model_id) and model_id not in image_classification_models:
        image_classification_models.append(model_id)

print(f"Number of ic models available for inference: {len(image_classification_models)}")

Number of ic models available for inference: 162


<IPython.core.display.Javascript object>

In [39]:
# display the model-ids in a dropdown to select a model for inference.
infer_model_dropdown = Dropdown(
    options=image_classification_models,
    value="pytorch-ic-alexnet",
    description="Select a model:",
    style={"description_width": "initial"},
    layout={"width": "max-content"},
)

<IPython.core.display.Javascript object>

In [40]:
display(infer_model_dropdown)

Dropdown(description='Select a model:', layout=Layout(width='max-content'), options=('pytorch-ic-alexnet', 'py…

<IPython.core.display.Javascript object>

In [41]:
print(infer_model_dropdown.value)

pytorch-ic-alexnet


<IPython.core.display.Javascript object>